In [1]:
import numpy as np
from fractions import gcd
from numbers import Number

import torch
from torch import nn
from torch.nn import functional as F

In [3]:
import os

import argparse
import numpy as np
import random
import sys
import time
import shutil
from importlib import import_module
from numbers import Number

import torch
from torch.utils.data import Sampler, DataLoader


from utils import Logger, load_pretrain
import matplotlib.pyplot as plt
from lanegcn import get_model


config, Dataset, collate_fn, net, loss, post_process, opt = get_model()

def worker_init_fn(pid):
    np_seed = int(pid)
    np.random.seed(np_seed)
    random_seed = np.random.randint(2 ** 32 - 1)
    random.seed(random_seed)

dataset = Dataset('./dataset/preprocess/train_crs_dist6_angle90.p', config, train=True)
train_loader = DataLoader(
        dataset,
        batch_size=config["batch_size"],
        num_workers=config["workers"],
        shuffle=False,   # True: At each epoch, reorder the data
        collate_fn=collate_fn,
        pin_memory=True,
        worker_init_fn=worker_init_fn,   # The next 36 were thrown away
        drop_last=True,
    )


from symbol import break_stmt


data = {}
for i, data in enumerate(train_loader):
    data = dict(data)
    break

In [7]:
from lanegcn import actor_gather
actors, actor_idcs = actor_gather(data["feats"])
print(actors.size())

torch.Size([528, 3, 20])


In [16]:
conv = nn.Conv1d(3, 32, kernel_size=3, padding=1, stride=1, bias=False)
out = conv(actors)
print(out.size())
conv1 = nn.Conv1d(32, 32, kernel_size=3, padding=1, stride=1, bias=False)
out1 = conv1(out)
print(out1.size())

relu = nn.ReLU(inplace=True)
out2 = relu(out1)
print(out2.size())

norm1 = nn.GroupNorm(1,32)
out3 = norm1(out2)
print(out3.size())



torch.Size([528, 32, 20])
torch.Size([528, 32, 20])
torch.Size([528, 32, 20])
torch.Size([528, 32, 20])
